# Perplexity playbook

<p>
Mal Minhas, v0.3<br>
15.01.24
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with an API called [`pplx-api`](https://blog.perplexity.ai/blog/introducing-pplx-api) which was introduced in October 2023.  See [here](https://docs.perplexity.ai/docs/getting-started) for a guide on how to set up an API key and get started with `pplx-api`.  The following examples assume you have set up the API key in an environment variable called `PERPLEXITY_API_KEY`.

Supported models are [as follows](https://docs.perplexity.ai/docs/model-cards):
```
Model	                  Context Length       Model Type
codellama-34b-instruct	      16384          Chat Completion
llama-2-70b-chat               4096          Chat Completion
mistral-7b-instruct [2]	       4096 [1]      Chat Completion
mixtral-8x7b-instruct	       4096 [1]      Chat Completion
pplx-7b-chat	               8192          Chat Completion
pplx-70b-chat	               4096          Chat Completion
pplx-7b-online	               4096          Chat Completion
pplx-70b-online	               4096          Chat Completion

[1] We will be increasing the context length of mistral-7b-instruct to 32k tokens (see roadmap).
[2] This model refers to the v0.2 release of mistral-7b-instruct.
```

Of these models, the response [here](https://docs.perplexity.ai/discuss/65b27c5fc46f38007527c07f) suggests you should use `pplx-7b-online` or `pplx-70b-online` for real-time data.

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics including a utility function to render text for display in Jupyter:

In [2]:
import os
import requests
from IPython.display import Markdown, display

def renderCompletion(text: str):
    display(Markdown(text))
    
API_KEY = os.environ.get("PERPLEXITY_API_KEY")
DEFAULT_MODEL = "mistral-7b-instruct"
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = "Be precise and concise."
MID_LEVEL_DETAIL = "Explain in a few sentences."
GREAT_DETAIL = "Explain in a lot of detail."
POLITE_DETAIL = "You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user."
DEFAULT_TEMP = 0.0

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [12]:
def getPerplexityResponse(question:str, model:str=DEFAULT_MODEL, direction:str=MID_LEVEL_DETAIL, max_tokens:int=DEFAULT_TOKENS, temperature:float=DEFAULT_TEMP) -> str:
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": f"{direction}"
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    r = requests.post(url, json=payload, headers=headers)
    assert(r.status_code == 200)
    return renderCompletion(r.json().get("choices")[0].get("message").get("content"))

Let's test the function on an example query using the default Mistral 7b model and then Llama2.  Note the differences in response timing:

In [13]:
%%time

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
getPerplexityResponse(prompt)

In Southampton, UK, if you have a broken washing machine, you should call a professional appliance repair technician. They have the necessary skills and expertise to diagnose and fix common washing machine issues. You can look up local appliance repair services online or check directories like Yell or Thomson Local. Alternatively, you could contact a local appliance repair company, such as Southampton Appliance Repairs or A1 Appliance Repairs, for assistance. Make sure to describe the problem accurately when you call to help them prepare for the repair visit.

CPU times: user 30.9 ms, sys: 9.89 ms, total: 40.8 ms
Wall time: 1.28 s


In [14]:
%%time

getPerplexityResponse(prompt, model='llama-2-70b-chat')

If you have a broken washing machine in Southampton, UK, you can call a local appliance repair service or a professional washing machine technician to diagnose and fix the issue. You can search online for appliance repair services in your area or check online directories such as Yell or Checkatrade to find a reputable technician. Additionally, you can also contact the manufacturer's customer service number to see if they can provide assistance or recommend a repair service.

CPU times: user 33.3 ms, sys: 3.25 ms, total: 36.5 ms
Wall time: 2.33 s


In [15]:
%%time

getPerplexityResponse(prompt, model='codellama-34b-instruct', direction=POLITE_DETAIL)

I'm so sorry to hear that your washing machine is broken! There are a few options you can consider to get it mended.

1. Contact the manufacturer: If you have purchased the washing machine from a reputable brand, you can contact their customer service department to inquire about repair options. They may be able to provide you with information on authorized repair centers or repair kits that can help you fix the machine yourself.
2. Look for local appliance repair services: There may be local appliance repair services in your area that specialize in fixing washing machines. You can search online or check with local plumbers or electricians to find a reputable repair service.
3. Check with your insurance provider: If you have home insurance, you may be able to file a claim for the repair costs. Contact your insurance provider to see if they cover appliance repairs and what the process is for filing a claim.
4. Consider purchasing a new washing machine: If the repair costs are too high or you are unable to find a reliable repair service, it may be more cost-effective to purchase a new washing machine.

In terms of the trade you need to call, it depends on the type of repair you need. If the repair is minor and you can fix it yourself, you may not need to call any trade. However, if the repair requires more extensive work, you may need to call a plumber or an appliance repair technician.

I hope this information helps! If you have any further questions, please don't hesitate to ask.

CPU times: user 33.1 ms, sys: 3.73 ms, total: 36.8 ms
Wall time: 4.87 s


In [16]:
%%time

getPerplexityResponse(prompt, model='pplx-7b-online', direction=POLITE_DETAIL)

If you have a broken washing machine in Southampton, UK, you have a few options to get it repaired or disposed of:

1. **Hire a professional repair service**: You can search for local washing machine repair services on websites like Angi. These services can help you diagnose the issue and provide a quote for the repair. Keep in mind that the service charge may be around $100 to $150.

2. **Attempt a DIY repair**: If you're handy and the issue is minor, you might be able to fix the washing machine yourself. However, it's essential to be cautious when working with electrical wiring and appliance parts.

3. **Sell the broken appliance**: You can try selling the broken washing machine for scrap metal or parts. Some used appliance stores may pay for a non-working washer, and you can also list the parts on platforms like eBay.

4. **Donate or recycle**: If your washing machine is still in decent working order, you can donate it to a local charity or recycle it through a scrap metal yard or appliance store.

Remember to be cautious when handling appliances and consider hiring a professional if you're unsure about the repair process.

CPU times: user 24.4 ms, sys: 6.51 ms, total: 30.9 ms
Wall time: 4.11 s


Let's see how it handles a query with a more detailed response:

In [18]:
%%time

prompt = """"I have a broken garden gate which is no longer closing properly.  
What trade do I need to call to fix it?"""
getPerplexityResponse(prompt, model='pplx-7b-online', direction=GREAT_DETAIL)

To fix a broken garden gate, you would typically need to call a handyman or a carpenter. These professionals are skilled in repairing and maintaining various types of structures, including gates. They can assess the damage, identify the cause of the problem, and provide a solution to fix the gate.

Here are some steps to follow when dealing with a broken garden gate:

1. **Inspect the gate**: Examine the gate to determine the extent of the damage. Look for any visible cracks, loose hinges, or broken parts.

2. **Identify the cause**: Determine what caused the gate to break. It could be due to wear and tear, accidental damage, or a manufacturing defect.

3. **Call a handyman or carpenter**: If you're not comfortable fixing the gate yourself, or if the damage is extensive, it's best to call a professional. A handyman or carpenter can assess the situation and provide a quote for the repair.

4. **Get a quote**: Request a quote from the handyman or carpenter for the repair work. Make sure to ask about the cost, the time required for the repair, and any additional services they may offer.

5. **Schedule the repair**: Once you've received a quote and are satisfied with the terms, schedule the repair. Make sure to communicate any specific requirements or preferences you have for the repair.

6. **Monitor the progress**: Keep an eye on the progress of the repair work to ensure that it's being done correctly and to your satisfaction.

7. **Test the gate**: After the repair is complete, test the gate to ensure it's functioning properly. If there are any issues, communicate them to the handyman or carpenter for further adjustments.

Remember that the cost of the repair will depend on the extent of the damage and the materials needed for the repair. It's essential to get a quote from a professional to get an accurate estimate of the cost.

CPU times: user 26.2 ms, sys: 11.9 ms, total: 38 ms
Wall time: 2.77 s


## 3. Streaming interface

tbd

In [8]:
%%time

def getPerplexityStreamingResponse(question, model=DEFAULT_MODEL, direction=MID_LEVEL_DETAIL, max_tokens=DEFAULT_TOKENS, temperature=DEFAULT_TEMP):
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": True,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": direction,
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    print('start')
    response_stream = requests.post(url, json=payload, headers=headers)
    print('end')
    for response in response_stream:
        print(len(response))
    #return response.get("choices")[0].get("message").get("content")

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
#getPerplexityStreamingResponse(prompt, direction=GREAT_DETAIL)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 3.81 µs
